# Возьмем готовую разбивку номенклатур по категориям

In [3]:
import pandas
import numpy as np
import codecs
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn import cross_validation, grid_search, linear_model, metrics

colnames = ['name', 'id', 'class']
data = pandas.read_csv('nameCatIdCatName.csv', names=colnames, encoding='utf8')
data.dropna()
names = data['name'].tolist()
classes = data['class'].tolist()

# Найдем параметры регресии

In [4]:
vectorizer =  TfidfVectorizer()
corpus = vectorizer.fit_transform(names)
clf = LogisticRegressionCV(n_jobs=-1, max_iter=100, refit=True)
clf.fit(corpus, classes)

LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=-1, penalty='l2', random_state=None,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0)

# Построим по ним модель и проверим

In [ ]:
train_data, test_data, train_labels, test_labels = \
cross_validation.train_test_split(names,  classes, test_size = 0.3, random_state = 0)

In [ ]:
clf = LogisticRegression(C=10.0, class_weight='balanced', dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=-1, penalty='l2', random_state=None,
           solver='lbfgs', tol=0.0001)
log = Pipeline([('vectorizer', TfidfVectorizer()),
                      ('classifier', clf)])
log.fit(train_data, train_labels)

In [ ]:
score = log.score(test_data, np.array(test_labels
print(score)

# Полученная точность 93%

In [ ]:
# log_params = {
# 'classifier__solver' : ['newton-cg', 'sag', 'lbfgs'],
# 'classifier__max_iter' : [100, 200],
# 'classifier__C': [0.001, 0.1, 1, 10, 100],
# 'classifier__class_weight' : ['balanced', None],
# 'classifier__refit' : [False, True],
# 'classifier__multi_class' : ['ovr', 'multinomial']
# }
# cv = cross_validation.StratifiedShuffleSplit(train_labels, n_iter = 10, test_size = 0.2, random_state = 0)
# grid_cv = grid_search.GridSearchCV(log, log_params, scoring = 'accuracy', cv = cv)
# grid_cv.fit(train_data, train_labels)
# print grid_cv.best_score_
# print grid_cv.best_params_
# 0.854

# Раз уж мы тут все собрались, то посмотрим, что все это значит для наших  номенклатур

In [11]:
content = codecs.open("nom3utf.csv", 'r', 'utf8')
lines = [x.replace("\\", "") for x in content.readlines()]
real_test = np.random.choice(np.array(lines), size = 5)

# Логистическая регрессия:

In [12]:
predicts = log.predict_proba(real_test)
for x, predict in zip(real_test, predicts):
    print(x)
    for p, l in zip(predict, log.classes_):
        if (p > 0.3):
            print(p)
            print(l)
    print('\n')

Антеовин таб №63

0.45753367607
ЛЕКАРСТВА ПО РЕЦЕПТАМ


Сурепка трава 50г пак. Компания Хорст

0.72223899925
НАРОДНЫЕ СРЕДСТВА


Ницерголин лиоф. д/р-ра д/ин. 4мг 5мл фл. Синтез

0.824071952967
ЛЕКАРСТВА ПО РЕЦЕПТАМ


Белоснежная кожа скраб осветляющ 75мл Биокон

0.674227863664
УХОД ЗА ЛИЦОМ


Стикс целебная грязь крем дневной 50мл 750

0.862438474454
АРОМАТЕРАПИЯ




# Ну, не так уж и плохо